In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
import numpy as np
import pickle

class EmbeddingModel:
    def _init_(self, dim=32):
        self.vectorizer = TfidfVectorizer(stop_words='english')
        self.pca = PCA(n_components=dim)
        self.dim = dim

    def fit_article_embeddings(self, df_articles):
        tfidf = self.vectorizer.fit_transform(df_articles["title"])
        reduced = self.pca.fit_transform(tfidf.toarray())
        self.article_emb = {
            row.article_id: reduced[i] 
            for i, row in df_articles.iterrows()
        }

    def build_user_embeddings(self, df_interactions):
        user_emb = {}
        for user, group in df_interactions.groupby("user_id"):
            vectors = []
            for _, row in group[group.clicked == 1].iterrows():
                if row.article_id in self.article_emb:
                    vectors.append(self.article_emb[row.article_id])
            user_emb[user] = np.mean(vectors, axis=0) if vectors else np.zeros(self.dim)
        self.user_emb = user_emb

    def save(self, path="models/embed_model.pkl"):
        with open(path, "wb") as f:
            pickle.dump(self, f)